<a href="https://colab.research.google.com/github/KVijay2/DeepLearning-PyTorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline


In [2]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [3]:
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [4]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

for i, (X_Train, y_train) in enumerate(train_data):
  break

X_Train.shape

torch.Size([1, 28, 28])

In [5]:
x = X_Train.view(1,1,28,28)

x = F.relu(conv1(x))
x.shape

torch.Size([1, 6, 26, 26])

In [6]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 6, 13, 13])

In [7]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

In [8]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 16, 5, 5])

In [9]:
((28-2) / 2 - 2) / 2

5.5

In [10]:

# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Fully Connected Layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride 2
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride 2

    # Re-View to flatten it out
    X = X.view(-1, 16*5*5) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [11]:
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results
    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')


Epoch: 0  Batch: 600  Loss: 0.1623624861240387
Epoch: 0  Batch: 1200  Loss: 0.1641543209552765
Epoch: 0  Batch: 1800  Loss: 0.5098981857299805
Epoch: 0  Batch: 2400  Loss: 0.1306418627500534
Epoch: 0  Batch: 3000  Loss: 0.005703817121684551
Epoch: 0  Batch: 3600  Loss: 0.46332210302352905
Epoch: 0  Batch: 4200  Loss: 0.004197881557047367
Epoch: 0  Batch: 4800  Loss: 0.0018000779673457146
Epoch: 0  Batch: 5400  Loss: 0.07375213503837585
Epoch: 0  Batch: 6000  Loss: 0.0003859388525597751
Epoch: 1  Batch: 600  Loss: 0.004290326032787561
Epoch: 1  Batch: 1200  Loss: 0.2521086633205414
Epoch: 1  Batch: 1800  Loss: 0.002427826402708888
Epoch: 1  Batch: 2400  Loss: 0.0021776421926915646
Epoch: 1  Batch: 3000  Loss: 0.02223074808716774
Epoch: 1  Batch: 3600  Loss: 0.6111965179443359
Epoch: 1  Batch: 4200  Loss: 0.016707444563508034
Epoch: 1  Batch: 4800  Loss: 0.0006908098584972322
Epoch: 1  Batch: 5400  Loss: 0.0002799260546453297
Epoch: 1  Batch: 6000  Loss: 0.4848875403404236
Epoch: 2  Batc